In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn import metrics

import time
import os
import cv2

img_height = 64
img_width = 64
batch_size = 32
tileSize = 64
overlap = 20

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import os

class Tile:   
    x = 0
    y = 0
    width = 0
    height = 0
    imagex = 0
    imagey = 0
    roi = 0

    def __init__(self,x, y, overlap, width, height, imagex, imagey):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
#         self.imagex = x * (width - overlap)
#         self.imagey = y * (height - overlap)
        self.imagex = imagex
        self.imagey = imagey

    def populate(self,master):
        y = self.imagey
        x = self.imagex
        height = self.height
        width = self.width
        #print(y,x)
        self.roi = master[y:y+height, x:x+width]
        if (self.roi.shape[1] < width):
            #print("error", self.roi.shape, x)
            new_x = x - (width-self.roi.shape[1])
            self.roi = master[y:y+height, new_x:new_x+width]
            self.imagex = new_x


        if (self.roi.shape[0] <height):
            #print("error", self.roi.shape)
            new_y = y - (height-self.roi.shape[0])
            self.roi = master[new_y:new_y+height, x:x+width]
            self.imagey = new_y
        

def tileImage(image,width, height, overlap):
    t_width = width
    t_height = height
    overlap = overlap

    # get dimensions of image
    dimensions = image.shape
 
    # height, width, number of channels in image
    height = image.shape[0]
    width = image.shape[1]
    channels = image.shape[2]

    tilesInX = math.ceil(width / (t_width - overlap))
    tilesInY = math.ceil(height / (t_height - overlap))
    
    tiles = []

    
    for y in range(tilesInY):
        row = []
        for x in range(tilesInX):
            tempx =  x * (t_width - overlap)
            tempy = y * (t_height - overlap)
            
            if(y == tilesInY - 1):
                tempy = height - t_height
                
            if(x == tilesInX-1):
                tempx = width - t_width

                

            
            tile = Tile(x,y,overlap,t_width,t_height, tempx, tempy)
            tile.populate(image)
            row.append(tile)
        tiles.append(row)
        
    return tiles
        

In [3]:
def cnnInspect(image):
    tiles = tileImage(image,tileSize,tileSize,overlap)

    imarray = []
    
    imDefects = []

    for row in tiles:
        for tile in row:
            # convert cv2 to tensor flow
            start = time.perf_counter()
            tileIm = tile.roi
            tileImRGB = cv2.cvtColor(tileIm, cv2.COLOR_BGR2RGB)

            imarray.append(tileImRGB)
 
        arg = tf.convert_to_tensor(imarray, dtype=tf.float32)
        # predictions = curModel.predict(mainTensor)
        predictions = model.predict(arg)
        indices = np.where(predictions < 0.5)

    for i in indices[0]:
        rowNum = int(i/len(tiles[0]))
        rowIndent = i%len(tiles[0])
        imDefects.append(tiles[rowNum][rowIndent].roi)
        
    return imDefects

In [4]:
model = keras.models.load_model("inspection_model")


In [5]:
normalNames = os.listdir("personTester/images/normal/")

print(len(normalNames))

normalIms = []

for name in normalNames:
    normalIms.append(cv2.imread("personTester/images/normal/"+name))


8


In [6]:
defectNames = os.listdir("personTester/images/defects/")

print(len(defectNames))

defectIms = []

for name in defectNames:
    defectIms.append(cv2.imread("personTester/images/defects/"+name))


8


In [7]:
correctDefect = 0
incorrectDefect = 0

print("defcets")

start = time.perf_counter()

for image in defectIms:
    imdefects = cnnInspect(image)
    print(len(imdefects))
    if len(imdefects)==0:
        incorrectDefect = incorrectDefect +1
    else:
        correctDefect = correctDefect + 1
        
correctNormal = 0
incorrectNormal = 0
print("normals")

for image in normalIms:
    imdefects = cnnInspect(image)
    print(len(imdefects))
    if len(imdefects)==0:
        incorrectNormal = incorrectNormal +1
    else:
        correctNormal = correctNormal + 1
        
taken = time.perf_counter() - start


defcets
44
0
0
2
1
0
3
101
normals
0
0
1
0
0
0
0
0


In [8]:
print("Correct Defect:", correctDefect)
print("incorrect Defect:", incorrectDefect)
print("Correct Normal:", correctNormal)
print("incorrect Normal:", incorrectNormal)
acc = ((correctDefect+correctDefect) / (correctDefect+incorrectDefect+correctNormal+incorrectNormal))*100
print("accracy:", acc)
print("time taken:", taken)

Correct Defect: 5
incorrect Defect: 3
Correct Normal: 1
incorrect Normal: 7
accracy: 62.5
time taken: 338.4484490000177
